# I. Design the problem
We're getting profits from the ##number_5_orders## order, yet the rentention rate untill this order is ##rentetion_rate##. The most significant drop is at ##numbers_of_order## at.
Therefore, we want to identify **the prospensity churn of the ##number_of_order##**

**Problem**: Which customers at the order ##number of orders## will churn?

**Reason**: If this customers doesn't churn at this order, our business could start making profit out of this single customer. 

**Baseline**: If we don't have any action when the model correctly predict patients who retain, the focus is shifted to how accurate the model predicts customers who churn. Through the business value analysis, for all records who is predicted as churn, 56% (Given the assumptions are correct) 

Note: Prospensity model could be used for any behavior

# Setup

In [ ]:
PROJECT_ID = "YOUR-PROJECT-ID" 
REGION = 'US'

In [ ]:
!pip install google-cloud-bigquery
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)  

# II. Collect Data

In [ ]:
%%bigquery --project $PROJECT_ID df

SELECT 
    *
FROM
  `firebase-public-project.analytics_153293282.events_*`

In [ ]:
df.info()

# III. Prepare Data

### Static 
- Age
- State
- Start Weight/IBM
- Motivation
### Dynamic
- Number of interactions they make via the app last month 
- Number of tickets sent to our team 
- Tracker quiz
- Weight Loss Rate
- Current medication
- Current dose 
- Current subscription fee 

### 1. Prepare label

Gather all patients who already has n orders, who cohorted from August 2024, and have the n orders before October: 
- Churn patients are the one who paused their subscription before the next order is fired.
- Retain patients who are the one didn't paused their subscription before the next order is fired.

In [ ]:
%%bigquery --project $PROJECT_ID

In [ ]:
CREATE OR REPLACE VIEW <name> AS (
    
)

### 2. Extract Initial Data

### 3. Extract Journey Data

### 4. Combine the Data Together

# IV. Training the prospensity model

Hyperparameter tuning here: https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create

### 1. XGBoost

In [ ]:
# %%bigquery --project $PROJECT_ID

# CREATE OR REPLACE MODEL bqmlga4.churn_xgb

# OPTIONS(
#   MODEL_TYPE="BOOSTED_TREE_CLASSIFIER",
#   INPUT_LABEL_COLS=["churned"]
# ) AS

# SELECT
#   * EXCEPT(user_pseudo_id)
# FROM
#   bqmlga4.train

### 2. Neural Network Model

In [ ]:
# %%bigquery --project $PROJECT_ID

# CREATE OR REPLACE MODEL bqmlga4.churn_dnn

# OPTIONS(
#   MODEL_TYPE="DNN_CLASSIFIER",
#   INPUT_LABEL_COLS=["churned"]
# ) AS

# SELECT
#   * EXCEPT(user_pseudo_id)
# FROM
#   bqmlga4.train

# V. Model Evaluation

In [ ]:
%%bigquery --project $PROJECT_ID

SELECT
  *
FROM
  ML.EVALUATE(MODEL bqmlga4.churn_logreg)

In [ ]:
%%bigquery --project $PROJECT_ID

SELECT
  expected_label,
  _0 AS predicted_0,
  _1 AS predicted_1
FROM
  ML.CONFUSION_MATRIX(MODEL bqmlga4.churn_logreg)

In [ ]:
%%bigquery df_roc --project $PROJECT_ID
SELECT * FROM ML.ROC_CURVE(MODEL bqmlga4.churn_logreg)

In [ ]:
df_roc.plot(x="false_positive_rate", y="recall", title="AUC-ROC curve")

# VI. Making Prediction

In [ ]:
%%bigquery --project $PROJECT_ID

SELECT
  *
FROM
  ML.PREDICT(MODEL bqmlga4.churn_logreg,
  (SELECT * FROM bqmlga4.train)) 

In [ ]:
%%bigquery --project $PROJECT_ID

SELECT
  user_pseudo_id,
  churned,
  predicted_churned,
  predicted_churned_probs[OFFSET(0)].prob as probability_churned
  
FROM
  ML.PREDICT(MODEL bqmlga4.churn_logreg,
  (SELECT * FROM bqmlga4.train))

# VII. Serving Model

In [ ]:
%%bigquery df --project $PROJECT_ID

SELECT
  user_pseudo_id,
  churned,
  predicted_churned,
  predicted_churned_probs[OFFSET(0)].prob as probability_churned
  
FROM
  ML.PREDICT(MODEL bqmlga4.churn_logreg,
  (SELECT * FROM bqmlga4.train))

Important: How to deploy this model through BigQuery ML